In [ ]:
# this script analyzes how much CosMx protein binds to 
# non-tissue regions, requires raw protein images
import pandas as pd
import numpy as np
from skimage.io import imread, imshow
import os

In [ ]:
# this function formats the number such that we can have
# a list of FOVs up to 4 digits
def format_numbers(numbers):
    formatted_numbers = []
    for num in numbers:
        num_str = str(num)
        if len(num_str) <= 3:
            formatted_num = f"FOV{num_str.zfill(3)}"
        else:
            formatted_num = f"FOV{num_str}"
        formatted_numbers.append(formatted_num)
    return formatted_numbers

In [ ]:
# I already had all the TMA FOVs from a previous analysis
# all you have to do is find non tissue FOVs
# this block of code finds non tissue FOVs from TMA FOVs
a1 = list(range(817, 817+7))+list(range(845, 845+7))+list(range(873, 873+7))+list(range(901, 901+7))+list(range(929, 929+7))+list(range(957, 957+7))+list(range(789, 789+7))
a2 = list(range(797, 797+8))+list(range(825, 825+8))+list(range(853, 853+8))+list(range(881, 881+8))+list(range(909, 909+8))+list(range(937, 937+8))+list(range(965, 965+8))+list(range(993, 993+8))
a3 = list(range(777, 777+8))+list(range(805, 805+8))+list(range(833, 833+8))+list(range(861, 861+8))+list(range(889, 889+8))+list(range(917, 917+8))
b1 = list(range(56, 56+7))+list(range(83, 83+7))+list(range(110, 110+7))+list(range(137, 137+7))+list(range(164, 164+7))+list(range(191, 191+7))+list(range(218, 218+7))
b2 = list(range(38, 38+7))+list(range(65, 65+7))+list(range(92, 92+7))+list(range(119, 119+7))+list(range(146, 146+7))+list(range(173, 173+7))+list(range(200, 200+7))+list(range(227, 227+7))
b3 = list(range(46, 46+8))+list(range(73, 73+8))+list(range(100, 100+8))+list(range(127, 127+8))+list(range(154, 154+8))+list(range(181, 181+8))+list(range(208, 208+8))
c1 = list(range(271, 271+8))+list(range(298, 298+8))+list(range(325, 325+8))+list(range(352, 352+8))+list(range(379, 379+8))+list(range(406, 406+8))+list(range(433, 433+8))
c2 = list(range(280, 280+8))+list(range(307, 307+8))+list(range(334, 334+8))+list(range(361, 361+8))+list(range(388, 388+8))+list(range(415, 415+8))+list(range(442, 442+8))+list(range(469, 469+8))
c3 = list(range(288, 288+8))+list(range(315, 315+8))+list(range(342, 342+8))+list(range(369, 369+8))+list(range(396, 396+8))+list(range(423, 423+8))+list(range(450, 450+8))+list(range(477, 477+8))
d1 = list(range(514, 514+8))+list(range(541, 541+8))+list(range(568, 568+8))+list(range(595, 595+8))+list(range(622, 622+8))+list(range(649, 649+8))+list(range(676, 676+8))
d2 = list(range(523, 523+7))+list(range(550, 550+7))+list(range(577, 577+7))+list(range(604, 604+7))+list(range(631, 631+7))+list(range(658, 658+7))+list(range(685, 685+7))
d3 = list(range(531, 531+8))+list(range(558, 558+8))+list(range(585, 585+8))+list(range(612, 612+8))+list(range(639, 639+8))+list(range(666, 666+8))+list(range(693, 693+8))+list(range(720, 720+8))
PANCProteinTMAS = [a1,a2,a3,b1,b2,b3,c1,c2,c3,d1,d2,d3]
PANCProteinTMAS = [item for sublist in PANCProteinTMAS for item in sublist]
print(len(PANCProteinTMAS))

allfovs = list(range(1,1009))
antiTMAs = [x for x in allfovs if x not in PANCProteinTMAS]
print((antiTMAs))

antiTMAsNames = format_numbers(antiTMAs)
print(antiTMAsNames)

In [ ]:
# for each of the non-tissue region FOVs, this block of code
# finds the average expression and stores all of it as a 
# numpy matrix
gene_snr_mat = []
nameConverter = pd.read_csv('./ProteinIDProteinNameGene.csv')
proteinnames = np.sort(nameConverter['Protein ID'].values)
# iterate through each gene
for i in range(len(proteinnames)):
    # iterate through each fov
    gene_snr = []
    for j in range(len(antiTMAsNames)):
        protein_image_path = '/mnt/scratch1/Luke/CosMXTest/PANC_Protein_RAW/SAHA_Protein_PANC_A1_WCMCTP_1_13124_HC_22424/20240224_224444_S1/AnalysisResults/n5gubpribt/'+antiTMAsNames[j]+'/ProteinImages/'
        filenames = np.sort(os.listdir(protein_image_path))
        try:
            file = [item for item in filenames if proteinnames[i] in item][0]
        except IndexError:
            file = None
        if file:
            image = imread(protein_image_path + file)
            gene_snr.append(np.sum(image)/(np.shape(image)[0]*np.shape(image)[1]))
        else:
            gene_snr.append(0)
    gene_snr_mat.append(gene_snr)
print(gene_snr_mat)
np.save('matrix.npy', gene_snr_mat)

In [ ]:
# for each of the non-tissue region FOVs, this block of code
# finds the standard deviation for expression and stores 
# all of it as a numpy matrix
# std helps see if we have a uniform protein binding or
# actually fallen tissue
gene_snr_mat = []
nameConverter = pd.read_csv('./ProteinIDProteinNameGene.csv')
proteinnames = np.sort(nameConverter['Protein ID'].values)
# iterate through each gene
for i in range(len(proteinnames)):
    # iterate through each fov
    gene_snr = []
    for j in range(len(antiTMAsNames)):
        protein_image_path = '/mnt/scratch1/Luke/CosMXTest/PANC_Protein_RAW/SAHA_Protein_PANC_A1_WCMCTP_1_13124_HC_22424/20240224_224444_S1/AnalysisResults/n5gubpribt/'+antiTMAsNames[j]+'/ProteinImages/'
        filenames = np.sort(os.listdir(protein_image_path))
        try:
            file = [item for item in filenames if proteinnames[i] in item][0]
        except IndexError:
            file = None
        if file:
            image = imread(protein_image_path + file)
            gene_snr.append(np.std(image))
        else:
            gene_snr.append(0)
    gene_snr_mat.append(gene_snr)
print(gene_snr_mat)
np.save('matrix_std.npy', gene_snr_mat)

In [ ]:
# this block of code generates boxplots of mean expression for each gene
# higher value means higher noise or more expression in non-tissue regions
import matplotlib.pyplot as plt 
loaded_matrix = np.transpose(np.load('matrix.npy'))
print(np.shape(loaded_matrix))

noise = np.mean(loaded_matrix, axis=0)

for i in range(len(noise)):
    loaded_matrix[:, i] = loaded_matrix[:, i]-noise[i]

j=0
protein_image_path = '/mnt/scratch1/Luke/CosMXTest/PANC_Protein_RAW/SAHA_Protein_PANC_A1_WCMCTP_1_13124_HC_22424/20240224_224444_S1/AnalysisResults/n5gubpribt/'+antiTMAsNames[j]+'/ProteinImages/'
filenames = np.sort(os.listdir(protein_image_path))

nameConverter = pd.read_csv('./ProteinIDProteinNameGene.csv')

proteinnames = []
for filename in filenames:
    temp = filename.copy()
    temp = temp.replace('.TIF', '')
    temp = temp.split('_')[5]
    if temp in nameConverter['Protein ID'].values:
        proteinname = nameConverter.loc[nameConverter['Protein ID'] == temp, 'Protein Name'].values[0]
        proteinnames.append(proteinname)
    else:
        proteinnames.append(temp)
print(proteinnames)

plt.figure(figsize=(25, 6))
plt.boxplot(loaded_matrix, labels=proteinnames)
plt.xlabel('Genes')
plt.xticks(rotation=45)
plt.ylabel('Mean Expression')
plt.title('Protein "Stickiness" using Mean Expression in Empty FOVs')
plt.grid(False)
plt.show()

In [ ]:
# this block of code generates boxplots of standard deviation for each gene
loaded_matrix_std = np.transpose(np.load('matrix_std.npy'))
print(np.shape(loaded_matrix_std))

plt.figure(figsize=(25, 6))
plt.boxplot(loaded_matrix_std, labels=proteinnames)
plt.xlabel('Genes')
plt.xticks(rotation=45)
plt.ylabel('Mean Expression')
plt.title('Protein "Stickiness" using Standard Deviation in Empty FOVs')
plt.grid(False)
plt.show()

In [ ]:
# examine some genes of interest from looking at the boxplots above
genes_of_interest = ['CD68', 'SMA', 'Fibronectin', 'CD31', 'CD39', 'ICOS', 'PD-L1', 'pan-RAS', 'EpCAM', 'GZMB']
gene_indices = [proteinnames.index(element) for element in genes_of_interest]
print(gene_indices)
for i in range(len(gene_indices)):
    single_gene = list(loaded_matrix[:, gene_indices[i]])
    sorted_list = sorted(single_gene, reverse=True)
    top5val = sorted_list[:5]
    top5ind = [single_gene.index(val) for val in top5val]
    top5fov = []
    for j in (top5ind):
        top5fov.append(antiTMAsNames[j])
    # top5fov = sorted(top5fov)
    print('Gene = '+genes_of_interest[i])
    print(top5fov)
    print(top5val)

In [ ]:
# map plot of where the expressions are and high much std is there
# great way to visualize whether or not you're seeing tissue
# or noise
df_FOV = pd.read_csv('PANCProteinFOVs.csv').drop('Slide', axis=1)
df_FOV = df_FOV[df_FOV['FOV'].isin(antiTMAs)]

for i in range(len(gene_indices)):
    df_FOV[genes_of_interest[i]] = loaded_matrix[:, gene_indices[i]]
    df_FOV[genes_of_interest[i]+'_std'] = loaded_matrix_std[:, gene_indices[i]]

print(df_FOV)

x_min = df_FOV['X_mm'].min()
x_max = df_FOV['X_mm'].max()

for i in range(len(gene_indices)):
    plt.figure()
    plt.scatter(df_FOV['X_mm'], df_FOV['Y_mm'], c=df_FOV[genes_of_interest[i]], s=(df_FOV[genes_of_interest[i]+'_std']*1.2)+1, cmap='viridis') 
    plt.xlabel('X_mm')
    plt.ylabel('Y_mm')
    plt.colorbar(label=genes_of_interest[i]+' Value')
    plt.title('FOVs for: '+genes_of_interest[i])
    plt.axis('equal')
    plt.xlim(x_min, x_max)
    plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()
    plt.show()

In [ ]:
i = 0
j = 0
nameConverter = pd.read_csv('./ProteinIDProteinNameGene.csv')
proteinnames = nameConverter['Protein ID'].values
protein_image_path = '/mnt/scratch1/Luke/CosMXTest/PANC_Protein_RAW/SAHA_Protein_PANC_A1_WCMCTP_1_13124_HC_22424/20240224_224444_S1/AnalysisResults/n5gubpribt/'+antiTMAsNames[j]+'/ProteinImages/'
filenames = np.sort(os.listdir(protein_image_path))
print(filenames)

'''print(proteinnames[i])
file = [item for item in filenames if proteinnames[i] in item]
print(file)

image = imread(protein_image_path + file)
print(image)'''

In [ ]:
# create a list of strings from 'FOV001' to 'FOV052'
fov_list = ['FOV' + str(i).zfill(3) for i in range(1, 53)]

gene_snr_mat = []
# iterate over the fovs
for j in range(len(fov_list)):
    protein_image_path = '/mnt/scratch1/Luke/CosMXProteinStickiness/APE_Protein_RAW_FOV_SPOT/SAHA_APE_A9_3_WCM_CTP_4_121923_HCStJude_020824/20240209_022201_S1/AnalysisResults/o540m5n1ny/'+fov_list[j]+'/ProteinImages/'
    filenames = np.sort(os.listdir(protein_image_path))
    # load the cell mask image
    fov_num = fov_list[j].replace('FOV', '')
    cell_mask = imread('/mnt/scratch1/Luke/CosMXProteinStickiness/APE_Protein_RAW_FOV_SPOT/SAHA_APE_A9_3_WCM_CTP_4_121923_HCStJude_020824/20240209_022201_S1/CellStatsDir/'+fov_list[j]+'/CellLabels_F'+fov_num+'.tif')
    # initialize the gene SNR list
    gene_snr = []
    # load protein images and process them
    for i in range(len(filenames)):
        # load protein image
        image = imread(protein_image_path + filenames[i])
        # create masks for cell and empty pixels
        cell_pixels = (cell_mask > 0)
        empty_pixels = (cell_mask == 0)
        # extract pixel values for cell and empty regions using masks
        pixels_cell = image[cell_pixels]
        pixels_empty = image[empty_pixels]
        # calculate mean values for cell and empty regions
        mean_cell = np.mean(pixels_cell)
        mean_empty = np.mean(pixels_empty)
        # calculate gene SNR and append to the list
        gene_snr.append(mean_cell / mean_empty)
    gene_snr_mat.append(gene_snr)

In [ ]:
import matplotlib.pyplot as plt 

df = pd.read_csv('GeneSNRMatrix.csv')
df = df.to_numpy()
df = -np.log(df)

nameConverter = pd.read_csv('./ProteinIDProteinNameGene.csv')

proteinnames = []
for filename in filenames:
    temp = filename.copy()
    temp = temp.replace('.TIF', '')
    temp = temp.split('_')[5]
    if temp in nameConverter['Protein ID'].values:
        proteinname = nameConverter.loc[nameConverter['Protein ID'] == temp, 'Protein Name'].values[0]
        proteinnames.append(proteinname)
    else:
        proteinnames.append(temp)
print(proteinnames)

# df = pd.DataFrame(gene_snr_mat)

plt.figure(figsize=(25, 6))
plt.boxplot(df, labels=proteinnames)
plt.xlabel('Genes')
plt.xticks(rotation=45)
plt.ylabel('Negative Log SNR')
plt.title('Protein "Stickiness" using SNR')
plt.grid(False)
plt.show()

# df.to_csv('GeneSNRMatrix.csv', sep=',', index=False)

In [ ]:
# find the max expression value for each protein
# create a list of strings from 'FOV001' to 'FOV052'
fov_list = ['FOV' + str(i).zfill(3) for i in range(1, 53)]

max_val_mat = []
# iterate over the fovs
for j in range(len(fov_list)):
    protein_image_path = '/mnt/scratch1/Luke/CosMXProteinStickiness/APE_Protein_RAW_FOV_SPOT/SAHA_APE_A9_3_WCM_CTP_4_121923_HCStJude_020824/20240209_022201_S1/AnalysisResults/o540m5n1ny/'+fov_list[j]+'/ProteinImages/'
    filenames = np.sort(os.listdir(protein_image_path))
    # initialize the gene SNR list
    max_val = []
    # load protein images and process them
    for i in range(len(filenames)):
        # load protein image
        image = imread(protein_image_path + filenames[i])

        max_val.append(image.max())
    max_val_mat.append(max_val)

In [ ]:
max_val = np.max(max_val_mat, axis=0)
df_max_val = pd.DataFrame(max_val)

In [ ]:
df_max_val.index = proteinnames
df_max_val.columns = ['Max Expression Level']
print(df_max_val)

In [ ]:
df_max_val.to_csv('MaxExpressionEachGene.csv', index=True)